A股（至少是指数）是否有周内效应呢？就是特定周几盈利高一些，让我们来验证一下吧。

In [1]:
import pandas as pd
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
from datetime import datetime
import json
import numpy as np
import cnfundutils
import libtrdb2
import trading2_pb2
import tradingdb2_pb2
import tradingdb2_pb2_grpc

import plotly.express as px

isStaticImg = True

cnfundutils.init2()

trdb2cfg = libtrdb2.loadConfig('./trdb2.yaml')

FUNDRESULTS_PATH = './calcfundfull.csv'
dfFundResults = cnfundutils.loadFundResult(FUNDRESULTS_PATH)

FUNDS_PATH = './funds.xlsx'
dfFunds = cnfundutils.loadFundBasicInfo(FUNDS_PATH)

In [2]:
paramsaip = trading2_pb2.AIPParams(
    money=10000,
    type=trading2_pb2.AIPTT_MONTHDAY,
    day=1,
)    

paramsinit = trading2_pb2.InitParams(
    money=10000,
)

buyup = trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['up'],
    strVals=['ema.31'],
)

buydown = trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['down'],
    strVals=['ema.31'],
)

lststart = [1, 2, 3, 4, 5]
lstend = [2, 3, 4, 5, 1]
lsttitle = ['周一', '周二', '周三', '周四', '周五']
lstparams = []
assets = ['cnfunds.510310']
for day in range(0, 4):
    for i in range(0, 5):
        buy0 = trading2_pb2.CtrlCondition(
            name='weekday',
            vals=[lststart[i]],
        )

        sell0 = trading2_pb2.CtrlCondition(
            name='weekday',
            vals=[libtrdb2.nextWeekDay(lststart[i], day + 1)],
        )

        paramsbuy = trading2_pb2.BuyParams(
            perHandMoney=1,
        )

        paramssell = trading2_pb2.SellParams(
            perVolume=1,
        )
        
        s0 = trading2_pb2.Strategy(
            name="normal",
            asset=libtrdb2.str2Asset(assets[0]),
        )

        s0.buy.extend([buy0])
        s0.sell.extend([sell0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell)
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trading2_pb2.SimTradingParams(
            assets=[libtrdb2.str2Asset('cnfunds.510310')],
            startTs=0,
            endTs=-1,
            strategies=[s0],
            title='{}持有{}天'.format(lsttitle[i], day+1)
        ))
        
lstpnlaip = libtrdb2.simTradings(trdb2cfg, lstparams)

# baseline    
buy0 = trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trading2_pb2.InitParams(
    money=10000,
)

pnl = libtrdb2.simTradingEx3(trdb2cfg, assets, 0, -1, [buy0], None, paramsbuy, None, paramsinit, None)
lstpnlaip.append({'title': 'baseline', 'pnl': pnl})

dfaip = libtrdb2.buildPNLReport(lstpnlaip)

In [3]:
dfaip[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance', 'perWinRate']].sort_values(by='totalReturns', ascending=False)

,title,maxDrawdown,maxDrawdownStart,maxDrawdownEnd,totalReturns,sharpe,annualizedReturns,annualizedVolatility,variance,perWinRate
13,周四持有3天,0.38407,2015-06-08,2016-01-28,3.57772,1.66038,0.34072,0.18714,0.42818,0.60497
18,周四持有4天,0.43381,2015-06-08,2016-01-28,3.12894,1.22132,0.28140,0.20584,0.30938,0.57692
9,周五持有2天,0.27994,2015-07-13,2016-02-01,3.11149,1.63623,0.27909,0.15224,0.28848,0.62185
14,周五持有3天,0.35112,2015-06-30,2015-09-15,2.62284,1.05749,0.21450,0.17447,0.19011,0.58101
8,周四持有2天,0.28010,2015-06-12,2016-01-15,2.31682,0.87624,0.17405,0.16440,0.13802,0.55143
17,周三持有4天,0.42543,2015-06-08,2016-02-29,2.30736,0.66803,0.17280,0.21377,0.12941,0.56098
20,baseline,0.46014,2015-06-08,2016-01-28,2.07778,0.49605,0.14596,0.23377,0.10860,0.00000
19,周五持有4天,0.34311,2015-06-08,2015-08-26,1.86573,0.41975,0.11443,0.20114,0.09089,0.56077
4,周五持有1天,0.21515,2015-06-24,2015-09-18,1.79130,0.57852,0.10459,0.12894,0.04299,0.60000
0,周一持有1天,0.18081,2015-06-30,2016-02-01,1.67765,0.65396,0.09177,0.09446,0.03362,0.00000


这里的胜率是卖出价高于买入价的比例，0.5左右，其实应该就是一个正常随机游走的状态

特别是只考虑周内买卖的情况下，胜率几乎决定了盈利与否

周5的胜率较高，但周5的盈利却不算高，应该是周五涨幅不够高造成的

In [4]:
paramsaip = trading2_pb2.AIPParams(
    money=10000,
    type=trading2_pb2.AIPTT_MONTHDAY,
    day=1,
)    

paramsinit = trading2_pb2.InitParams(
    money=10000,
)

buyup = trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['up'],
    strVals=['ema.30'],
)

buydown = trading2_pb2.CtrlCondition(
    name='indicatorsp',
    operators=['down'],
    strVals=['ema.30'],
)

lststart = [1, 2, 3, 4, 5]
lstend = [2, 3, 4, 5, 1]
lsttitle = ['周一', '周二', '周三', '周四', '周五']
lstparams = []
assets = ['cnfunds.510310']
for day in range(0, 4):
    for i in range(0, 5):
        buy0 = trading2_pb2.CtrlCondition(
            name='weekday',
            vals=[lststart[i]],
        )

        sell0 = trading2_pb2.CtrlCondition(
            name='weekday',
            vals=[libtrdb2.nextWeekDay(lststart[i], day + 1)],
        )

        paramsbuy = trading2_pb2.BuyParams(
            perHandMoney=1,
        )

        paramssell = trading2_pb2.SellParams(
            perVolume=1,
        )
        
        s0 = trading2_pb2.Strategy(
            name="normal",
            asset=libtrdb2.str2Asset(assets[0]),
        )

        s0.buy.extend([buy0, buyup])
        s0.sell.extend([sell0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell)
        s0.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trading2_pb2.SimTradingParams(
            assets=[libtrdb2.str2Asset('cnfunds.510310')],
            startTs=0,
            endTs=-1,
            strategies=[s0],
            title='{}持有{}天up'.format(lsttitle[i], day+1)
        ))
        
        s1 = trading2_pb2.Strategy(
            name="normal",
            asset=libtrdb2.str2Asset(assets[0]),
        )

        s1.buy.extend([buy0, buydown])
        s1.sell.extend([sell0])
        s1.paramsBuy.CopyFrom(paramsbuy)
        s1.paramsSell.CopyFrom(paramssell)
        s1.paramsInit.CopyFrom(paramsinit)        
        lstparams.append(trading2_pb2.SimTradingParams(
            assets=[libtrdb2.str2Asset('cnfunds.510310')],
            startTs=0,
            endTs=-1,
            strategies=[s1],
            title='{}持有{}天down'.format(lsttitle[i], day+1)
        ))        
        
lstpnlaip = libtrdb2.simTradings(trdb2cfg, lstparams)

# baseline    
buy0 = trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trading2_pb2.InitParams(
    money=10000,
)

pnl = libtrdb2.simTradingEx3(trdb2cfg, assets, 0, -1, [buy0], None, paramsbuy, None, paramsinit, None)
lstpnlaip.append({'title': 'baseline', 'pnl': pnl})

dfaip = libtrdb2.buildPNLReport(lstpnlaip)

In [5]:
dfaip[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance', 'perWinRate']].sort_values(by='totalReturns', ascending=False)

,title,maxDrawdown,maxDrawdownStart,maxDrawdownEnd,totalReturns,sharpe,annualizedReturns,annualizedVolatility,variance,perWinRate
36,周四持有4天up,0.14483,2019-04-19,2020-04-24,3.27488,2.00485,0.30069,0.13502,0.45847,0.59434
26,周四持有3天up,0.11126,2013-10-21,2014-04-21,3.22143,2.19024,0.29362,0.12036,0.34467,0.62085
16,周四持有2天up,0.10801,2015-11-09,2017-02-03,2.77911,1.93042,0.23516,0.10628,0.23711,0.56039
34,周三持有4天up,0.19493,2015-11-09,2016-06-24,2.69413,1.32055,0.22393,0.14685,0.25442,0.57658
28,周五持有3天up,0.17844,2018-01-24,2019-01-14,2.60550,1.59982,0.21221,0.11389,0.24507,0.59024
18,周五持有2天up,0.12824,2018-01-23,2019-01-14,2.39117,1.54361,0.18388,0.09969,0.15549,0.61765
8,周五持有1天up,0.08569,2014-12-29,2015-01-23,2.21394,1.55135,0.16046,0.08409,0.10878,0.62814
40,baseline,0.46014,2015-06-08,2016-01-28,2.07778,0.49605,0.14596,0.23377,0.10860,0.00000
38,周五持有4天up,0.27455,2017-11-22,2019-01-14,1.87907,0.64598,0.11619,0.13343,0.13642,0.54854
24,周三持有3天up,0.26851,2015-06-12,2016-06-24,1.86672,0.60573,0.11456,0.13960,0.08589,0.54378


这里由于还考虑了均线，所以情况要复杂一些